In [1]:
import cx_Oracle
import pandas as pd

usr ='movies_project'
pwd = 'intekglobal123'

#tns="db:1521/xe"
dsn = cx_Oracle.makedsn("db", 1521, service_name="xepdb1")

connection = cx_Oracle.connect(usr, pwd, dsn,encoding="UTF-8")


#check connectivity
cursor = connection.cursor()
cursor.execute("""   SELECT * FROM DUAL """)
for val in cursor:
    print("Values:", val)

Values: ('X',)


In [2]:
def q_max_names(colname):
    '''Select max number of names in all columns
    '''
    return f"""  
SELECT MAX(num_{colname}) FROM  (
SELECT  (LENGTH( "{colname}" ) - LENGTH(REPLACE( "{colname}" ,',','')) + 1) AS num_{colname} 
FROM (
SELECT TO_CHAR(TRIM("{colname}" )) AS "{colname}"   FROM ROTTEN_TOMATOES_RAW rtr 
WHERE "{colname}"  IS NOT NULL 
)
)
"""

In [3]:
q_column_names ="""
SELECT table_name, column_name, data_type, data_length
FROM USER_TAB_COLUMNS
WHERE table_name = 'ROTTEN_TOMATOES_RAW'
"""

cursor.execute(q_column_names)
dtypes_dic = {row[1]:row[2] for row in cursor.execute(q_column_names)}
dtypes_dic

{'RELEASE_DATE_STREAMING': 'CLOB',
 'BOX_OFFICE_GROSS_USA': 'CLOB',
 'RUNTIME': 'CLOB',
 'PRODUCTION_CO': 'CLOB',
 'SOUND_MIX': 'CLOB',
 'VIEW_THE_COLLECTION': 'CLOB',
 'SYNOPSIS': 'CLOB',
 'CRITICS_CONSENSUS': 'CLOB',
 'TOMATOMETER_SCORE': 'CLOB',
 'TOMATOMETER_TOTAL_COUNT': 'CLOB',
 'AUDIENCE_SCORE': 'CLOB',
 'AUDIENCE_USER_RATINGS': 'CLOB',
 'ASPECT_RATIO': 'CLOB',
 'YEAR': 'NUMBER',
 'FILM_NAME': 'CLOB',
 'RATING': 'CLOB',
 'GENRE': 'CLOB',
 'ORIGINAL_LANGUAGE': 'CLOB',
 'DIRECTOR': 'CLOB',
 'PRODUCER': 'CLOB',
 'WRITER': 'CLOB',
 'RELEASE_DATE_THEATERS': 'CLOB'}

In [11]:
multi_valued_cols = ['DIRECTOR','PRODUCER','WRITER', 'PRODUCTION_CO','GENRE']

In [12]:
#one_to_many_columns = ['DIRECTOR','RATING','PRODUCER','WRITER','ORIGINAL_LANGUAGE', 'PRODUCTION_CO']

In [13]:
#Max names
max_names_results =  [int(result[0]) for col in multi_valued_cols for result in cursor.execute(q_max_names(col))]

max_names_results

[5, 8, 8, 22, 6]

In [14]:
def q_split_comma_sep_vals(cols, pos_list):
    def q_select_f(col, pos):
        return f"""TRIM(TO_CHAR(regexp_substr("{col}", '[^,]+',1, {pos}))) AS {col}_{pos},"""

    q_select_cols = '\n'.join(
        [
            '\n'.join(
                [
                    q_select_f(col, pos)
                    for pos in range(1, num_pos + 1)
                ]
            ) for col,num_pos in  zip(cols, pos_list)
        ]
    )
    q_select_cols = q_select_cols[:-1]+'\n'

    return f"""SELECT
rtr.*,
{q_select_cols}
FROM ROTTEN_TOMATOES_RAW rtr 
"""

In [15]:
#Flatten multi_valued_cols 

In [16]:
q_create_flattened_view = f"""CREATE OR REPLACE VIEW ROTTEN_TOMATOES_FLAT AS
{q_split_comma_sep_vals(multi_valued_cols, max_names_results)}
"""
cursor.execute(q_create_flattened_view)

In [17]:
print(q_create_flattened_view)

CREATE OR REPLACE VIEW ROTTEN_TOMATOES_FLAT AS
SELECT
rtr.*,
TRIM(TO_CHAR(regexp_substr("DIRECTOR", '[^,]+',1, 1))) AS DIRECTOR_1,
TRIM(TO_CHAR(regexp_substr("DIRECTOR", '[^,]+',1, 2))) AS DIRECTOR_2,
TRIM(TO_CHAR(regexp_substr("DIRECTOR", '[^,]+',1, 3))) AS DIRECTOR_3,
TRIM(TO_CHAR(regexp_substr("DIRECTOR", '[^,]+',1, 4))) AS DIRECTOR_4,
TRIM(TO_CHAR(regexp_substr("DIRECTOR", '[^,]+',1, 5))) AS DIRECTOR_5,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 1))) AS PRODUCER_1,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 2))) AS PRODUCER_2,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 3))) AS PRODUCER_3,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 4))) AS PRODUCER_4,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 5))) AS PRODUCER_5,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 6))) AS PRODUCER_6,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 7))) AS PRODUCER_7,
TRIM(TO_CHAR(regexp_substr("PRODUCER", '[^,]+',1, 8))) AS PRODUCER_8,
TRIM(TO_CHAR(regexp_substr("W

In [18]:
#print(q_create_flattened_view)

In [ ]:
#cursor.execute(q_split_comma_sep_vals("WRITER",'1'))
for r in cursor.execute(q_split_comma_sep_vals("WRITER",'1')):
    print(r[0])